<a href="https://colab.research.google.com/github/deepankar27/ML_Tutorials/blob/master/NLP/findthecontextspan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is POC to find the span of a context given a sentence

In [6]:
!pip install transformers

In [7]:
import torch
from transformers import BertTokenizer, BertModel
from transformers import DistilBertTokenizer, DistilBertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

In [8]:
import numpy as np
import string
import torch.nn as nn


In [9]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [145]:
text_1 = "One of the main issues faced in the development projects was the delays faced in the projects due to missing information to complete certain tasks"

text_2 = "With this work, it is possible to build a map of associates and their work experience and also map different organisation units and list of work experience the unit holds"

In [146]:
marked_text_1 = "[CLS] " + text_1 + " [SEP]"
# Tokenize our sentence with the BERT tokenizer.
tokenized_text_1 = tokenizer.tokenize(marked_text_1)

# Map the token strings to their vocabulary indeces.
indexed_tokens_1 = tokenizer.convert_tokens_to_ids(tokenized_text_1)

# Mark each of the 22 tokens as belonging to sentence "1".
segments_ids_1 = [1] * len(tokenized_text_1)

# Convert inputs to PyTorch tensors
tokens_tensor_1 = torch.tensor([indexed_tokens_1])
segments_tensors_1 = torch.tensor([segments_ids_1])


In [147]:
marked_text_2 = "[CLS] " + text_2 + " [SEP]"
# Tokenize our sentence with the BERT tokenizer.
tokenized_text_2 = tokenizer.tokenize(marked_text_2)

# Map the token strings to their vocabulary indeces.
indexed_tokens_2 = tokenizer.convert_tokens_to_ids(tokenized_text_2)

segments_ids_2 = [1] * len(tokenized_text_2)

# Convert inputs to PyTorch tensors
tokens_tensor_2 = torch.tensor([indexed_tokens_2])
segments_tensors_2 = torch.tensor([segments_ids_2])

In [148]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor_1, segments_tensors_1)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states_1 = outputs[2]


In [149]:
hidden_states_1[0].shape

torch.Size([1, 27, 768])

In [150]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor_2, segments_tensors_2)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states_2 = outputs[2]


In [151]:
hidden_states_2[0].shape

torch.Size([1, 33, 768])

In [152]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

skiplist = {w: True for symbol in string.punctuation
            for w in [symbol, tokenizer.encode(symbol, add_special_tokens=False)[0]]}

def masking(input_ids):
    mask = [[(x not in skiplist) and (x != 0) for x in d] for d in input_ids.cpu().tolist()]
    return mask

In [153]:
def encode_document(txt):

    inputs = tokenizer(txt, padding='longest', truncation='longest_first',
                        return_tensors='pt', max_length=120)
    with torch.no_grad():
        #PP = bert(**inputs, output_hidden_states=True).hidden_states
        PP = model(inputs["input_ids"], attention_mask=inputs["attention_mask"])[0]
    
    #print(f"PP.shape {PP.shape}")
    linear = nn.Linear(768, 768, bias=False)

    PP = linear(PP)
    mask = torch.tensor(masking(inputs['input_ids'].to(DEVICE)), device=DEVICE).unsqueeze(2).float()
    D = PP * mask
    D = torch.nn.functional.normalize(D, p=2, dim=2)
    
    keep_dims = True

    if not keep_dims:
        D, mask = D.cpu().to(dtype=torch.float16), mask.cpu().bool().squeeze(-1)
        D = [d[mask[idx]] for idx, d in enumerate(D)]

    return D

In [154]:
token_embed_1 = encode_document(text_1)
#token_embed_1 = torch.tensor(token_embed_1).reshape(token_embed_1.shape[1],768)
print(token_embed_1.shape)

token_embed_2 = encode_document(text_2)
#token_embed_2 = torch.tensor(token_embed_2).reshape(token_embed_2.shape[1],768)
print(token_embed_2.shape)

torch.Size([1, 27, 768])
torch.Size([1, 33, 768])


In [155]:
(token_embed_1@token_embed_2.permute(0, 2, 1)).max(2).values.sum(1)[0].cpu().detach().numpy()

array(1.1913357, dtype=float32)